- unix commands
- git
- vagrant
    - unix server settings
    - iac of apache http server setup with vagrant provision (single file for full setup of website)
    - wordpress lamp stakc setup
    - apache tomcat server setup
- json vs yaml uses:
    - YAML (YAML Ain't Markup Language) is a human-readable data serialization standard commonly used for configuration files and data exchange between applications.(know writable)
    - JSON (JavaScript Object Notation) is a lightweight, text-based data interchange format that uses human-readable syntax to store and transmit data objects consisting of key-value pairs and arrays.(know readable)
- project:
    - nginx server , tomcat , nfs service(centralised storage) , rabbitMQ(queuing agent(message broker)), mssql , memcache(cache service for database), Elastic Search.
    - doing all these form vagrant is another part

## Infrastructure as Code (IaC)

**Environment Standardization**
- `Vagrantfile` as version-controlled infrastructure definition
- Consistent environments across development, staging, and testing
- Eliminates configuration drift between environments
- Infrastructure changes tracked through Git commits

**Multi-Environment Management**
```ruby
# Different configurations per environment
if ENV['VAGRANT_ENV'] == 'production'
  config.vm.box = "ubuntu/20.04"
  config.vm.provider.memory = 4096
else
  config.vm.box = "ubuntu/18.04" 
  config.vm.provider.memory = 2048
end
```

## CI/CD Pipeline Integration

**Automated Testing Infrastructure**
- Spin up clean environments for each test run
- `vagrant up && vagrant provision && vagrant destroy` in CI pipelines
- Parallel testing with multiple VMs
- Integration with Jenkins, GitLab CI, GitHub Actions

**Pipeline Commands**
```bash
# CI/CD integration examples
vagrant up --no-provision
vagrant provision --provision-with=test
vagrant ssh -c "run-tests.sh"
vagrant destroy -f
```

## Configuration Management Integration

**Ansible Integration**
```ruby
config.vm.provision "ansible" do |ansible|
  ansible.playbook = "playbooks/site.yml"
  ansible.inventory_path = "inventory/staging"
  ansible.limit = "all"
  ansible.extra_vars = {
    environment: "staging"
  }
end
```

**Chef/Puppet Support**
- Test cookbooks and manifests before production deployment
- Validate configuration changes in isolated environments
- Role-based provisioning for different server types

## Container Orchestration Testing

**Docker Integration**
```ruby
config.vm.provision "docker" do |d|
  d.build_image "/vagrant/app"
  d.run "app", 
    args: "-p 8080:8080 -v /vagrant:/app"
end
```

**Kubernetes Testing**
- Local K8s cluster simulation
- Test deployments before pushing to production
- Validate YAML manifests and Helm charts

## Advanced DevOps Features

**Multi-Machine Architectures**
```ruby
# Simulate production topology
config.vm.define "web" do |web|
  web.vm.box = "nginx-server"
  web.vm.network "private_network", ip: "192.168.50.10"
end

config.vm.define "db" do |db|
  db.vm.box = "mysql-server"
  db.vm.network "private_network", ip: "192.168.50.11"
end

config.vm.define "cache" do |cache|
  cache.vm.box = "redis-server"
  cache.vm.network "private_network", ip: "192.168.50.12"
end
```

## Monitoring & Logging Setup

**Observability Stack Testing**
- Deploy ELK stack locally
- Test Prometheus/Grafana configurations
- Validate log aggregation pipelines
- Monitor resource utilization patterns

**Health Check Integration**
```ruby
config.vm.provision "shell", inline: <<-SHELL
  # Install monitoring agents
  curl -L https://github.com/prometheus/node_exporter/releases/download/v1.0.1/node_exporter-1.0.1.linux-amd64.tar.gz | tar xz
  sudo cp node_exporter-1.0.1.linux-amd64/node_exporter /usr/local/bin/
SHELL
```

## Security & Compliance Testing

**Security Scanning**
- Test security configurations before deployment
- Validate firewall rules and network policies
- Compliance checking with tools like InSpec
- Vulnerability assessment in controlled environments

**Secret Management**
```ruby
# Test secret management tools
config.vm.provision "shell" do |s|
  s.inline = "curl -o vault.zip https://releases.hashicorp.com/vault/1.8.0/vault_1.8.0_linux_amd64.zip"
  s.inline = "unzip vault.zip && sudo mv vault /usr/local/bin/"
end
```

## Performance & Load Testing

**Load Testing Environments**
- Simulate production load conditions
- Test auto-scaling configurations
- Validate performance under stress
- Benchmark different configurations

**Resource Monitoring**
```ruby
config.vm.provider "virtualbox" do |vb|
  vb.customize ["modifyvm", :id, "--ioapic", "on"]
  vb.customize ["modifyvm", :id, "--memory", "4096"]
  vb.customize ["modifyvm", :id, "--cpus", "2"]
end
```

## DevOps Pipeline Automation

**Automated Deployment Testing**
```bash
#!/bin/bash
# deployment-test.sh
vagrant up
vagrant provision --provision-with=deploy
curl -f http://localhost:8080/health || exit 1
vagrant ssh -c "systemctl status myapp" || exit 1
vagrant destroy -f
```

**Blue-Green Deployment Simulation**
- Test deployment strategies locally
- Validate rollback procedures
- Database migration testing
- Zero-downtime deployment validation

## Cloud Provider Integration

**AWS/Azure/GCP Testing**
- Test cloud-init scripts locally
- Validate Terraform configurations
- Pre-flight checks for cloud deployments
- Cost optimization through local testing

**Hybrid Cloud Scenarios**
```ruby
# Test cloud connectivity
config.vm.provision "shell", inline: <<-SHELL
  aws configure set region us-west-2
  aws s3 ls # Test cloud connectivity
SHELL
```

## Operational Benefits

**Disaster Recovery Testing**
- Simulate server failures
- Test backup and restore procedures
- Validate monitoring alerts
- Practice incident response

**Documentation & Runbooks**
- Living documentation through code
- Automated environment setup instructions
- Repeatable deployment procedures
- Knowledge sharing through version control

**Cost Optimization**
- Local testing reduces cloud costs
- Faster feedback loops
- Reduced production debugging
- Efficient resource utilization testing

These DevOps-focused Vagrant capabilities enable teams to build robust, tested, and reliable infrastructure while maintaining development velocity and operational excellence.

---
---
# Server management:


### Top 3 Linux Server Distributions:

**1. Ubuntu - 33.9% (overall) / 19.8% (web servers)**
Ubuntu is the leading Linux distro, holding 33.9% of the Linux market, while for web servers specifically, Ubuntu accounts for 19.8% of the Linux market, making it the most popular Linux distribution. Linux is also most used for web servers, and the most common Linux distribution is Ubuntu, followed by Debian.

**2. Debian - 16% (overall) / 10.7% (web servers)**
Second place goes to Debian, which accounts for 16% of Linux-based websites, and another source shows Debian is second, powering 10.7% of Linux websites.

**3. CentOS - 9.3% (overall) / 4.0% (web servers)**
followed by CentOS with a share of 9.3% for websites, while another measurement shows CentOS is third with 4.0% of the market.

**Notable mentions:**
- **Red Hat Enterprise Linux (RHEL)** - 0.8% (though this may be underrepresented as many enterprise deployments aren't publicly visible)
- **Other distributions** like Fedora, SUSE, and Gentoo have much smaller shares below 1%

 

# Servers: Overview, Types & Setup Commands

## What are Servers?
Servers are computer programs or devices that provide functionality, services, or resources to other programs or devices (called clients) over a network. They listen for requests and respond with data or services.

## Most Used Server Types (Top 3)

**1. Web Servers (HTTP/HTTPS) - ~80% usage**
- **Purpose:** Serve web pages and web applications
- **Most Popular:** Apache HTTP Server, Nginx, Microsoft IIS
- **Use Cases:** Websites, web applications, API endpoints

**2. Database Servers - ~70% usage** 
- **Purpose:** Store, manage, and provide access to databases
- **Most Popular:** MySQL, PostgreSQL, MongoDB, Microsoft SQL Server
- **Use Cases:** Data storage, web app backends, business applications

**3. File/Storage Servers - ~60% usage**
- **Purpose:** Store and share files across networks
- **Most Popular:** Samba, NFS, FTP servers
- **Use Cases:** File sharing, backups, centralized storage

---

# Server Setup Commands in Linux

## 1. HTTP Web Server Setup

### Apache HTTP Server Setup:
```bash
# Install Apache
sudo apt update
sudo apt install apache2 -y

# Start and enable Apache
sudo systemctl start apache2
sudo systemctl enable apache2

# Check status
sudo systemctl status apache2

# Basic configuration
sudo ufw allow 'Apache Full'        # Allow HTTP/HTTPS through firewall
sudo chown -R www-data:www-data /var/www/html    # Set proper ownership

# Test - Create simple page
echo "<h1>Apache Server Running!</h1>" | sudo tee /var/www/html/index.html

# Access: http://your-server-ip
```

### Nginx Web Server Setup:
```bash
# Install Nginx
sudo apt update
sudo apt install nginx -y

# Start and enable Nginx
sudo systemctl start nginx
sudo systemctl enable nginx
sudo systemctl status nginx

# Firewall configuration
sudo ufw allow 'Nginx Full'

# Test configuration
sudo nginx -t

# Create test page
echo "<h1>Nginx Server Running!</h1>" | sudo tee /var/www/html/index.html

# Reload configuration
sudo systemctl reload nginx
```

## 2. Database Server Setup

### MySQL Database Server:
```bash
# Install MySQL
sudo apt update
sudo apt install mysql-server -y

# Start and enable MySQL
sudo systemctl start mysql
sudo systemctl enable mysql

# Secure installation
sudo mysql_secure_installation

# Login to MySQL
sudo mysql -u root -p

# Basic MySQL commands (inside MySQL shell)
CREATE DATABASE testdb;
CREATE USER 'testuser'@'localhost' IDENTIFIED BY 'password';
GRANT ALL PRIVILEGES ON testdb.* TO 'testuser'@'localhost';
FLUSH PRIVILEGES;
EXIT;

# Check status
sudo systemctl status mysql
```

### PostgreSQL Database Server:
```bash
# Install PostgreSQL
sudo apt update
sudo apt install postgresql postgresql-contrib -y

# Start and enable PostgreSQL
sudo systemctl start postgresql
sudo systemctl enable postgresql

# Access PostgreSQL
sudo -u postgres psql

# Create database and user (inside psql shell)
CREATE DATABASE testdb;
CREATE USER testuser WITH PASSWORD 'password';
GRANT ALL PRIVILEGES ON DATABASE testdb TO testuser;
\q

# Check status
sudo systemctl status postgresql
```

## 3. File Server Setup (Samba)

### Samba File Server:
```bash
# Install Samba
sudo apt update
sudo apt install samba samba-common-bin -y

# Create shared directory
sudo mkdir /home/shared
sudo chmod 777 /home/shared

# Configure Samba
sudo cp /etc/samba/smb.conf /etc/samba/smb.conf.backup
sudo nano /etc/samba/smb.conf

# Add to smb.conf file:
[shared]
    path = /home/shared
    browseable = yes
    writable = yes
    guest ok = yes
    read only = no

# Create Samba user
sudo smbpasswd -a $USER

# Start and enable Samba
sudo systemctl start smbd nmbd
sudo systemctl enable smbd nmbd

# Configure firewall
sudo ufw allow samba

# Check status
sudo systemctl status smbd
```

## Quick Server Management Commands

```bash
# Check all running services
sudo systemctl list-units --type=service --state=running

# View server logs
sudo journalctl -u apache2 -f     # Apache logs
sudo tail -f /var/log/nginx/error.log    # Nginx logs
sudo tail -f /var/log/mysql/error.log    # MySQL logs

# Restart services
sudo systemctl restart apache2
sudo systemctl restart nginx
sudo systemctl restart mysql

# Check open ports
sudo netstat -tlnp
sudo ss -tlnp
```


# LAMP Stack - Quick Overview

**LAMP** = **L**inux + **A**pache + **M**ySQL + **P**HP - a complete web development platform.

**How it works:** Linux runs the system → Apache serves web pages → PHP processes dynamic content → MySQL stores data.

**Quick Installation:**
```bash
sudo apt update && sudo apt install lamp-server^ -y
sudo mysql_secure_installation
echo "<?php phpinfo(); ?>" | sudo tee /var/www/html/info.php
```

**Test:** Visit `http://your-server-ip/info.php` to confirm LAMP is running.

**Why popular:** Free, stable, massive community support, powers most websites globally.

# WordPress - Quick Overview

**WordPress** is a **Content Management System (CMS)** - software that lets you create/manage websites without coding.

**Where it fits:** WordPress runs **ON TOP** of LAMP stack:
- **Linux** (OS) → **Apache** (web server) → **MySQL** (stores WordPress data) → **PHP** (runs WordPress code) → **WordPress** (CMS layer)

**Quick WordPress Installation on LAMP:**
```bash
# Download & extract WordPress
wget https://wordpress.org/latest.tar.gz
tar -xzf latest.tar.gz
sudo mv wordpress/* /var/www/html/
sudo chown -R www-data:www-data /var/www/html/
```

**Setup:** Create MySQL database → Visit your site → Follow WordPress 5-minute installer.

**Why popular:** Powers 43% of all websites, user-friendly, thousands of themes/plugins, no coding required.